In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
import os
from transformers import AutoTokenizer, T5ForConditionalGeneration
from PyPDF2 import PdfReader
from docx import Document

app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'docx'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB

os.makedirs(UPLOAD_FOLDER, exist_ok=True)

model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def extract_text_from_file(filepath):
    if filepath.endswith('.txt'):
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    elif filepath.endswith('.pdf'):
        reader = PdfReader(filepath)
        return '\n'.join([page.extract_text() for page in reader.pages])
    elif filepath.endswith('.docx'):
        doc = Document(filepath)
        return '\n'.join([para.text for para in doc.paragraphs])
    return None

def generate_summary(text):
    input_ids = tokenizer(
        [text],
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        max_length=400,
        return_tensors="pt"
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=3,
        num_beams=5,
        early_stopping=True
    )[0]

    return tokenizer.decode(output_ids, skip_special_tokens=True)

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)
            text = extract_text_from_file(filepath)
            if not text:
                return render_template('index.html', error="Не удалось извлечь текст")
            summary = generate_summary(text)
            os.remove(filepath)
            return render_template('index.html', summary=summary, original_text=text)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


Overwriting app.py


In [12]:
from flask import Flask
import threading
from google.colab.output import eval_js
import time

def run_flask():
    !python app.py

flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

time.sleep(2)

try:
    app_url = eval_js('google.colab.kernel.proxyPort(5000)')
    print(f"Приложение доступно по адресу: {app_url}")
except:
    print("Не удалось получить URL. Попробуйте следующие команды:")
    print("1. Нажмите 'Перезапустить среду выполнения' в меню 'Среда выполнения'")
    print("2. Запустите все ячейки снова")

  File "/content/app.py", line 88
    body { font-family: Arial, sans-serif; margin: 20px; }
                                                    ^
SyntaxError: invalid decimal literal
Приложение доступно по адресу: https://5000-m-s-a0pjpiqrikb8-a.us-west4-1.prod.colab.dev
